In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings(action='ignore')

data_dir = 'Week6_NN심화_Assignment/'

In [2]:
Train = pd.read_csv(data_dir + 'Train.csv')
Test = pd.read_csv(data_dir + 'Test.csv')
submission = pd.read_csv(data_dir + 'submission.csv')

In [3]:
X = Train.iloc[:, :-1].values
X = X.reshape(-1, 28, 28, 1)
y = Train.iloc[:, -1].values

X_test = Test.values
X_test = X_test.reshape(-1, 28, 28, 1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 16, stratify = y, shuffle = True)

# 정규화
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [5]:
BATCH_SIZE = 128
EPOCHS = 30

In [49]:
initializer = keras.initializers.HeUniform(seed = 21)
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, kernel_initializer=initializer),
    keras.layers.Dense(512, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, kernel_initializer=initializer),
    keras.layers.Dense(256, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, kernel_initializer=initializer),
    keras.layers.Dense(128, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)
])

print(model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_27 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)             

In [10]:
'''
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=2))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10 ,activation='softmax'))
'''

    
initializer = keras.initializers.HeUniform(seed = 21)
model = keras.Sequential([

    keras.layers.Conv2D(32,kernel_size=(3,3),input_shape=(28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(64,(3,3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),

    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),

    keras.layers.Dense(128, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.1),

    keras.layers.Dense(64, kernel_initializer=initializer),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),

    keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)
])




from keras.callbacks import ModelCheckpoint,EarlyStopping


early_stoping_callback = EarlyStopping(monitor='val_loss',patience=5,
                                      restore_best_weights=True)

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 batch_normalization_8 (Batc  (None, 26, 26, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_8 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 dropout_6 (Dropout)         (None, 26, 26, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 batch_normalization_9 (Batc  (None, 24, 24, 64)       256       
 hNormalization)                                      

In [28]:
#from keras.utils import multi_gpu_utils


optimizer = keras.optimizers.Adam(lr=0.01, decay = 0.001)

#model = multi_gpu_utils.multi_gpu_model(model,gpus=2)


model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [25]:
result = model.fit(X_train, y_train, epochs=EPOCHS, 
                   validation_data = (X_val, y_val), batch_size=BATCH_SIZE,
                  callbacks=[early_stoping_callback])

Epoch 1/30
372/372 [==============================] - 66s 177ms/step - loss: 0.3824 - accuracy: 0.8588 - val_loss: 0.5578 - val_accuracy: 0.7811
Epoch 2/30
372/372 [==============================] - 66s 178ms/step - loss: 0.2711 - accuracy: 0.8987 - val_loss: 0.4370 - val_accuracy: 0.8429
Epoch 3/30
372/372 [==============================] - 67s 180ms/step - loss: 0.2207 - accuracy: 0.9180 - val_loss: 0.2561 - val_accuracy: 0.9044
Epoch 4/30
372/372 [==============================] - 62s 166ms/step - loss: 0.1865 - accuracy: 0.9298 - val_loss: 0.2587 - val_accuracy: 0.9084
Epoch 5/30
372/372 [==============================] - 58s 156ms/step - loss: 0.1602 - accuracy: 0.9410 - val_loss: 0.2195 - val_accuracy: 0.9210
Epoch 6/30
372/372 [==============================] - 59s 159ms/step - loss: 0.1339 - accuracy: 0.9496 - val_loss: 0.2223 - val_accuracy: 0.9214
Epoch 7/30
372/372 [==============================] - 58s 155ms/step - loss: 0.1136 - accuracy: 0.9584 - val_loss: 0.2294 - val_ac

In [27]:
pred_label = np.argmax(model.predict(X_test), axis = 1)

submission['label'] = pred_label

submission.to_csv(data_dir + 'submission.csv', index = False)

submission.head()

,id,label
0,0,6
1,1,3
2,2,0
3,3,0
4,4,6
